<a href="https://colab.research.google.com/github/meti-94/CodingQuestions/blob/main/Predictive%20Analysis/Classic%20Classifiers/XGBoost-SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle -q
!pip install tensorflow-addons -q

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d williamscott701/memotion-dataset-7k

100% 695M/695M [00:24<00:00, 34.1MB/s]
100% 695M/695M [00:24<00:00, 30.2MB/s]


In [ ]:
!unzip -qq /content/memotion-dataset-7k.zip

In [ ]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
df = pd.read_csv('/content/memotion_dataset_7k/labels.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df = df.drop(columns = ['text_ocr'])
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [ ]:
full_df = df.copy()
full_df.isnull().any()

image_name           False
text_corrected        True
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [ ]:
clean_df = df.copy()
clean_df.dropna(inplace=True)
clean_df.isnull().any()

image_name           False
text_corrected       False
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [ ]:
df = df.replace({'humour': {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious':3},
            'sarcasm': {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2, 'very_twisted': 3},
            'offensive': {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3},
            'motivational': {'not_motivational': 0, 'motivational': 1},
            'overall_sentiment': {'very_negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very_positive': 4}})

In [ ]:
width = 100
height = 100
X_full = []
X_full_path = []
for i in tqdm(range(full_df.shape[0])):
    path = '/content/memotion_dataset_7k/images/'+full_df['image_name'][i]
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X_full.append(img)
    X_full_path.append(path)

 93%|█████████▎| 6499/6992 [00:46<00:04, 112.77it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 95%|█████████▌| 6662/6992 [00:47<00:01, 169.90it/s]/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [00:49<00:00, 139.96it/s]


In [ ]:
width = 100
height = 100
X_cleaned = []
X_cleaned_path = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X_cleaned.append(img)
        X_cleaned_path.append(path)

100%|██████████| 6987/6987 [00:45<00:00, 154.83it/s]


In [ ]:
deleted_paths = [paths for paths in X_full_path + X_cleaned_path if paths not in X_full_path or paths not in X_cleaned_path]
deleted_paths

['/content/memotion_dataset_7k/images/image_120.jpg',
 '/content/memotion_dataset_7k/images/image_4800.jpg',
 '/content/memotion_dataset_7k/images/image_6782.jpg',
 '/content/memotion_dataset_7k/images/image_6785.jpg',
 '/content/memotion_dataset_7k/images/image_6787.jpg',
 '/content/memotion_dataset_7k/images/image_6988.jpg',
 '/content/memotion_dataset_7k/images/image_6989.jpg',
 '/content/memotion_dataset_7k/images/image_6990.png',
 '/content/memotion_dataset_7k/images/image_6991.jpg',
 '/content/memotion_dataset_7k/images/image_6992.jpg']

In [ ]:
delete_var = [full_df, clean_df, X_cleaned, X_full, X_cleaned_path, X_full_path]
for i in delete_var:
    del i

In [ ]:
width = 100
height = 100
X = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)

100%|██████████| 6987/6987 [00:46<00:00, 149.71it/s]


In [ ]:
X = np.array(X)

In [ ]:
X.shape

(6982, 100, 100, 3)

In [ ]:
rows_to_drop = ['image_120.jpg',
              'image_4800.jpg',
              'image_6782.jpg',
              'image_6785.jpg',
              'image_6787.jpg',
              'image_6988.jpg',
              'image_6989.jpg',
              'image_6990.png',
              'image_6991.jpg',
              'image_6992.jpg']

In [ ]:
cleaner_df = df
cleaner_df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,3,1,0,0,4
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,0,1,0,1,4
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,2,0,0,0,3
3,image_4.png,10 Year Challenge - Sweet Dee Edition,2,2,2,1,3
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,3,2,0,2


In [ ]:
for images in rows_to_drop:
    cleaner_df.drop(cleaner_df[cleaner_df['image_name'] == images].index, inplace=True)

In [ ]:
len(cleaner_df)

6982

In [ ]:
cleaner_df.shape

(6982, 7)

In [ ]:
np.save('image_array', X)

In [ ]:
Y = cleaner_df.iloc[:,2:]
Y.shape

(6982, 5)

In [ ]:
X.shape, Y.shape

((6982, 100, 100, 3), (6982, 5))

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.resnet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
base_model_1 = tf.keras.applications.ResNet50(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')
base_model_2 = tf.keras.applications.VGG16(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
base_model_1.trainable = False
base_model_2.trainable = False

In [ ]:
global_average_layer = GlobalAveragePooling2D()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
del X

In [ ]:
image_input = tf.keras.Input(shape=(100, 100, 3), name = 'image_input')
iamge_layers = data_augmentation(image_input)
iamge_layers = preprocess_input(iamge_layers)
layer_bm_1 = base_model_1(image_input, training=False)
layer_bm_1 = Conv2D(2048, kernel_size=2,padding='valid')(layer_bm_1)
layer_bm_1 = Dense(512)(layer_bm_1)
layer_bm_2 = base_model_2(image_input, training=False)
layer_bm_2 = Dense(512)(layer_bm_2)
layers = tf.keras.layers.concatenate([layer_bm_1, layer_bm_2])
iamge_layers = global_average_layer(layer_bm_1)
image_layers = Dropout(0.2, name = 'dropout_layer')(iamge_layers)

In [ ]:
cleaner_df.shape

(6982, 7)

In [ ]:
def standardization(data):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\d+', '', x))
    data = data.apply(lambda x: re.sub(r'.com', '', x, flags=re.MULTILINE))
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

df['text_corrected'] = standardization(cleaner_df.text_corrected)

In [ ]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(cleaner_df.text_corrected, Y, test_size = 0.2, random_state=42)

In [ ]:
X_text_train

6568      how much money mark zuckerberg gave out imgflip
4004    crypto experts choose crypto investments based...
1011    me throwing away all the leonardo dicaprio mem...
4618    texan man searching for supplies and dry land ...
932     keep calm and use duct tape if you cant fix it...
                              ...                        
3773           let me tell you why thats bullshit imgflip
5193    and thats how mafia works the godfather mobile...
5228                              huh so babies cant swim
5392    leonard hofstadter a guy who actually made it ...
861     if you dont like me because im country just re...
Name: text_corrected, Length: 5585, dtype: object

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = np.asarray(X_text_train)
vectorize_layer.adapt(tf.convert_to_tensor(text_ds))

In [ ]:
embedding_dim=16

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
text_layers = vectorize_layer(text_input)
text_layers = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

text_layers = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True))(text_layers)

text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.GlobalMaxPooling1D()(text_layers)

# We add a vanilla hidden layer:
text_layers = tf.keras.layers.Dense(2048, activation="relu")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

In [ ]:
concatenate = tf.keras.layers.concatenate([image_layers, text_layers], axis=1)

In [ ]:
overall_layer = tf.keras.layers.Dense(2048, activation='softmax')(concatenate)

In [ ]:
prediction_layer_1 = tf.keras.layers.Dense(4, activation='softmax', name = 'sarcasm')
prediction_layer_2 = tf.keras.layers.Dense(4, activation='softmax', name = 'humuor')
prediction_layer_3 = tf.keras.layers.Dense(4, activation='softmax', name = 'offensive')
prediction_layer_4 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'motivational')
prediction_layer_5 = tf.keras.layers.Dense(5, activation='softmax', name = 'overall')

In [ ]:
output_1 = prediction_layer_1(overall_layer)
output_2 = prediction_layer_2(overall_layer)
output_3 = prediction_layer_3(overall_layer)
output_4 = prediction_layer_4(overall_layer)
output_5 = prediction_layer_5(overall_layer)
model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])

In [ ]:
import tensorflow_addons as tfa

In [ ]:
base_learning_rate = 0.01
losses = {
      "humuor": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "sarcasm": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "offensive": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "motivational": tf.keras.losses.BinaryCrossentropy(from_logits=False),
      "overall": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
}
lossWeights = {
      "humuor": 1.0, 
      "sarcasm": 1.0, 
      "offensive": 1.0, 
      "motivational": 1.0,
      "overall": 1.0
}
metric = {
    "humuor": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "sarcasm": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "offensive": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "motivational": ['acc',tfa.metrics.F1Score(num_classes=1, average="micro", threshold = 0.9)],
    "overall": ['acc',tfa.metrics.F1Score(num_classes=5, average="micro", threshold = 0.9)]
}


In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model-{epoch:03d}-{overall_accuracy:03f}-{val_overall_accuracy:03f}', save_format='tf', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=27, verbose=1, epsilon=1e-4, mode='min')

In [ ]:
learning_rates = [0.001] # [0.1, 0.01, 0.001, 0.0001, 0.00001]
for i in learning_rates:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])
    
    history = model.fit(x = {"image_input": X_train, "text": X_text_train},
                    y = {"sarcasm": y_train.sarcasm, 
                         "humuor": y_train.humour, 
                         "offensive": y_train.offensive, 
                         "motivational": y_train.motivational, 
                         "overall": y_train.overall_sentiment},
                    batch_size=64,
                    epochs=50,
                    verbose=2,
                    validation_split=0.2,
                    callbacks=[earlyStopping, mcp_save,reduce_lr_loss]
                   )
     

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


70/70 - 75s - loss: 6.2614 - sarcasm_loss: 1.3313 - humuor_loss: 1.3515 - offensive_loss: 1.3482 - motivational_loss: 0.6875 - overall_loss: 1.5429 - sarcasm_accuracy: 0.5000 - humuor_accuracy: 0.3498 - offensive_accuracy: 0.3915 - motivational_accuracy: 0.5962 - overall_accuracy: 0.4402 - val_loss: 6.1557 - val_sarcasm_loss: 1.3028 - val_humuor_loss: 1.3429 - val_offensive_loss: 1.3279 - val_motivational_loss: 0.6788 - val_overall_loss: 1.5034 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 75s/epoch - 1s/step
Epoch 2/50


70/70 - 59s - loss: 6.0494 - sarcasm_loss: 1.2821 - humuor_loss: 1.3272 - offensive_loss: 1.2990 - motivational_loss: 0.6733 - overall_loss: 1.4677 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.9851 - val_sarcasm_loss: 1.2609 - val_humuor_loss: 1.3278 - val_offensive_loss: 1.2908 - val_motivational_loss: 0.6664 - val_overall_loss: 1.4392 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 59s/epoch - 848ms/step
Epoch 3/50


70/70 - 58s - loss: 5.8954 - sarcasm_loss: 1.2462 - humuor_loss: 1.3119 - offensive_loss: 1.2622 - motivational_loss: 0.6636 - overall_loss: 1.4114 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.8614 - val_sarcasm_loss: 1.2305 - val_humuor_loss: 1.3192 - val_offensive_loss: 1.2640 - val_motivational_loss: 0.6579 - val_overall_loss: 1.3899 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 58s/epoch - 833ms/step
Epoch 4/50


...

70/70 - 59s - loss: 5.5728 - sarcasm_loss: 1.1750 - humuor_loss: 1.2913 - offensive_loss: 1.1735 - motivational_loss: 0.6515 - overall_loss: 1.2816 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6263 - val_sarcasm_loss: 1.1756 - val_humuor_loss: 1.3151 - val_offensive_loss: 1.2057 - val_motivational_loss: 0.6482 - val_overall_loss: 1.2817 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 59s/epoch - 841ms/step
Epoch 11/50


70/70 - 57s - loss: 5.5661 - sarcasm_loss: 1.1742 - humuor_loss: 1.2912 - offensive_loss: 1.1709 - motivational_loss: 0.6514 - overall_loss: 1.2784 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6228 - val_sarcasm_loss: 1.1752 - val_humuor_loss: 1.3154 - val_offensive_loss: 1.2048 - val_motivational_loss: 0.6482 - val_overall_loss: 1.2792 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 57s/epoch - 815ms/step
Epoch 12/50


70/70 - 58s - loss: 5.5615 - sarcasm_loss: 1.1737 - humuor_loss: 1.2911 - offensive_loss: 1.1692 - motivational_loss: 0.6515 - overall_loss: 1.2760 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6209 - val_sarcasm_loss: 1.1750 - val_humuor_loss: 1.3156 - val_offensive_loss: 1.2044 - val_motivational_loss: 0.6481 - val_overall_loss: 1.2776 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 58s/epoch - 823ms/step
Epoch 13/50


70/70 - 58s - loss: 5.5587 - sarcasm_loss: 1.1735 - humuor_loss: 1.2912 - offensive_loss: 1.1681 - motivational_loss: 0.6514 - overall_loss: 1.2745 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6198 - val_sarcasm_loss: 1.1750 - val_humuor_loss: 1.3157 - val_offensive_loss: 1.2043 - val_motivational_loss: 0.6481 - val_overall_loss: 1.2767 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 58s/epoch - 835ms/step
Epoch 14/50


70/70 - 56s - loss: 5.5568 - sarcasm_loss: 1.1733 - humuor_loss: 1.2911 - offensive_loss: 1.1673 - motivational_loss: 0.6515 - overall_loss: 1.2735 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6195 - val_sarcasm_loss: 1.1750 - val_humuor_loss: 1.3160 - val_offensive_loss: 1.2042 - val_motivational_loss: 0.6481 - val_overall_loss: 1.2762 - val_sarcasm_accuracy: 0.5067 - val_humuor_accuracy: 0.3366 - val_offensive_accuracy: 0.3474 - val_motivational_accuracy: 0.6491 - val_overall_accuracy: 0.4396 - lr: 0.0010 - 56s/epoch - 805ms/step
Epoch 15/50
70/70 - 30s - loss: 5.5555 - sarcasm_loss: 1.1732 - humuor_loss: 1.2912 - offensive_loss: 1.1669 - motivational_loss: 0.6514 - overall_loss: 1.2728 - sarcasm_accuracy: 0.5031 - humuor_accuracy: 0.3516 - offensive_accuracy: 0.3926 - motivational_accuracy: 0.6435 - overall_accuracy: 0.4481 - val_loss: 5.6196 - val_sarcasm_loss: 1.1750 - val

In [ ]:
best_model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])
best_model.load_weights('/content/model-014-0.448075-0.439570')
print('Best Model Loaded!')

Best Model Loaded!


In [ ]:
best_model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])

In [ ]:
evaluate = best_model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


44/44 - 5s - loss: 5.5555 - sarcasm_loss: 1.1835 - humuor_loss: 1.2856 - offensive_loss: 1.1656 - motivational_loss: 0.6425 - overall_loss: 1.2782 - sarcasm_accuracy: 0.4932 - humuor_accuracy: 0.3601 - offensive_accuracy: 0.4044 - motivational_accuracy: 0.6586 - overall_accuracy: 0.4495 - 5s/epoch - 114ms/step


In [ ]:
best_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 16)      160000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 16)      0           ['embedding[0][0]']        

In [ ]:
from keras.models import Model
embedder= Model(inputs=model.input, outputs=best_model.layers[-7].output)
embedder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 16)      160000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 16)      0           ['embedding[0][0]']        

In [ ]:
embedded_X_train = embedder.predict(x = {"image_input": X_train, "text": X_text_train},
                              batch_size=32,
                              verbose=2)
embedded_X_test = embedder.predict(x = {"image_input": X_test, "text": X_text_test},
                              batch_size=32,
                              verbose=2)
print(embedded_X_train.shape, embedded_X_test.shape)

175/175 - 13s - 13s/epoch - 75ms/step
44/44 - 2s - 2s/epoch - 55ms/step
(5585, 2560) (1397, 2560)


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       329
           1       0.38      0.77      0.51       503
           2       0.32      0.26      0.28       443
           3       0.00      0.00      0.00       122

    accuracy                           0.36      1397
   macro avg       0.17      0.26      0.20      1397
weighted avg       0.24      0.36      0.27      1397



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.16      0.19       335
           1       0.36      0.47      0.40       479
           2       0.32      0.38      0.35       440
           3       0.11      0.03      0.05       143

    accuracy                           0.32      1397
   macro avg       0.26      0.26      0.25      1397
weighted avg       0.29      0.32      0.30      1397

